In [91]:
import numpy as np
import pandas as pd
import os
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn import preprocessing
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from mpl_toolkits.basemap import Basemap
%run ./handler/gru_handler.ipynb

In [92]:
#!pip install cartopy
#!pip install shapely==1.8.5
#!pip install basemap

In [93]:
fields = ['VMAX','MSLP','LAT','LON','CD20','COHC','DTL','OAGE','RSST','U200','U20C','V20C','E000','RHLO','RHMD','RHHI',
          'PSLV','PSLV1','PSLV2','Z850','D200','REFC','PEFC','T000','R000', 'TWAC','G150','G200','G250',
          'V000','V850','V500','V300','TGRD','TADV','PENC','SHDC','SDDC','SHGC','DIVC','T150','T200','T250','SHRD',
          'SHTD','SHRS','SHTS','SHRG','PENV','VMPI','VMFX','HE05','O500','O700','CFLX']

class Norm():
    def __init__(self):
        self.scaler = preprocessing.MinMaxScaler()
    
    def normalize_data(self, df_train, df_val, df_test):
        tr = df_train.copy(deep=True)
        va = df_val.copy(deep=True)
        te = df_test.copy(deep=True)
        
        a = train_data[fields].values
        b = val_data[fields].values
        c = test_data[fields].values

        a = self.scaler.fit_transform(a)
        b = self.scaler.transform(b)
        c = self.scaler.transform(c)
        
        tr.loc[:, fields] = a
        va.loc[:, fields] = b
        te.loc[:, fields] = c
        return tr, va, te
    
    def unnormalize(self, data):
        scale = preprocessing.MinMaxScaler()
        scale.min_, scale.scale_ = self.scaler.min_[2:4], self.scaler.scale_[2:4]
        return scale.inverse_transform(data)


def split_data(data, storm_list, train_percent, val_percent, test_percent):
    storm_size = len(storm_list)
    train_size = int(storm_size * train_percent)
    train_ids = storm_list[:train_size]
    val_size = int(storm_size * val_percent)
    val_ids = storm_list[train_size:(train_size + val_size)]
    test_ids = storm_list[(train_size + val_size):]
    train_data = data.loc[data['ID'].isin(train_ids)]
    val_data = data.loc[data['ID'].isin(val_ids)]
    test_data = data.loc[data['ID'].isin(test_ids)]
    return train_data, val_data, test_data, train_ids, val_ids, test_ids

def get_model_data(storm_ids, data):
    # Getting 
    st_labels = []
    st_data = []
    storms_labels = []
    storms_data = []
    
    for storm in storm_ids:
        df_temp = data.loc[data.ID == storm]
        df_temp = df_temp.sort_values(by=['DATETIME'])
        df_temp.reset_index(drop=True, inplace=True)
        # Create df_temp copy for storm labels
        df_temp2 = df_temp.copy(deep=True)
        # Shift left Latitude and Longitude by 1 Time Step from Index 2 for labels
#         df_temp.iloc[1:, [8, 9]] = df_temp.iloc[:-1, [8, 9]].values
        df_temp2.iloc[:-1, [8, 9]] = df_temp2.iloc[1:, [8, 9]].values
        
        for x in range(len(df_temp)):
            storm_series = df_temp.loc[x, fields]
            storm_location = df_temp2.loc[x, ['LAT', 'LON']]
            st_data.append(storm_series)
            st_labels.append(storm_location)
        for y in range(89 - len(st_data)):
            zeros = [0] * 55
            zeros_labels = [0] * 2
            st_data.append(zeros)
            st_labels.append(zeros_labels)
        storms_data.append(st_data)
        storms_labels.append(st_labels)
        st_data = []
        st_labels = []
        
    storms_data = np.asarray(storms_data)
    storms_labels = np.asarray(storms_labels)
    return storms_data, storms_labels

def map_hurricane(plt_title):
    plt.figure(figsize=(20, 16))
    hm = Basemap(llcrnrlon=-100.,llcrnrlat=0.,urcrnrlon=30.,urcrnrlat=57.,projection='lcc',lat_1=20.,lat_2=40.,lon_0=-60.,resolution ='l',area_thresh=1000.)    
    hm.drawcoastlines()
    hm.drawparallels(np.arange(10,70,10),labels=[1,1,0,0])
    hm.drawmeridians(np.arange(-100,0,10),labels=[0,0,0,1])
    plt.suptitle(plt_title, fontsize=64)
    return hm

In [115]:
class GRU_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(GRU_Model, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, 1, self.hidden_dim).requires_grad_().cpu()



        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0)

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out


In [116]:
# Step 1: Load Data
df_data = pd.read_csv('/home/phoenix/Desktop/Cyclone_YOLO/Cyclone-Trajectory-Prediction (copy 1)/data/IO_out.csv')

In [117]:
pd.set_option('display.max_columns', None)
df_data.head()
df_data.loc[df_data.ID == 'IO042017']

,ID,NAME,DATETIME,YEAR,REGION,VMAX,MSLP,TYPE,LAT,LON,CD20,COHC,DTL,OAGE,RSST,U200,U20C,V20C,E000,RHLO,RHMD,RHHI,PSLV,PSLV1,PSLV2,Z850,D200,REFC,PEFC,T000,R000,TLAT,TLON,TWAC,G150,G200,G250,V000,V850,V500,V300,TGRD,TADV,PENC,SHDC,SDDC,SHGC,DIVC,T150,T200,T250,SHRD,SHTD,SHRS,SHTS,SHRG,PENV,VMPI,VMFX,HE05,O500,O700,CFLX,CATEGORY
1371,IO042017,IO04,2017-12-08 06:00:00,2017,Atlantic,40,993.0,1,172,863,107,35,256,0,274,-114,-154,134,3403,73,69,58,578,5.0,23.0,68,167,-14,-6,263,63,166,863,87.0,-2,1,6,49,89,56,17,39,15,129,130,320,350,167.0,-668,-520,-399,155,337,106,47,267,109,99,129,87,-83,-77,-20,0
1372,IO042017,IO04,2017-12-08 12:00:00,2017,Atlantic,45,989.0,1,177,864,108,34,221,50,272,-109,-132,146,3407,71,65,54,622,5.0,18.0,53,121,-10,-6,257,67,173,865,85.0,-2,1,5,41,86,50,1,41,13,122,136,321,315,124.0,-668,-520,-399,190,333,106,49,259,91,96,137,41,-42,-34,61,0
1373,IO042017,IO04,2017-12-08 18:00:00,2017,Atlantic,40,999.0,1,181,865,109,33,190,87,270,-73,-108,160,3380,69,62,48,622,5.0,23.0,51,81,-1,-6,244,70,183,865,89.0,-1,2,5,33,74,34,-9,15,4,148,137,332,271,73.0,-667,-523,-401,200,346,116,59,290,113,91,176,22,-26,-33,105,0
1374,IO042017,IO04,2017-12-09 00:00:00,2017,Atlantic,35,996.0,1,184,866,110,31,164,119,263,-22,-87,140,3360,66,57,42,660,5.0,18.0,42,68,-2,-6,235,72,191,869,94.0,-2,1,4,37,90,21,-23,37,14,148,93,341,341,59.0,-670,-525,-403,176,2,121,73,306,108,77,178,60,-36,-37,136,0
1375,IO042017,IO04,2017-12-09 06:00:00,2017,Atlantic,30,1000.0,1,189,869,110,34,128,110,261,24,-24,201,3402,67,56,39,664,10.0,21.0,39,69,0,-6,259,65,186,870,83.0,-2,1,3,50,72,14,-25,33,5,139,135,349,303,68.0,-669,-523,-405,171,11,87,82,289,120,73,109,58,-46,-60,78,-1
1376,IO042017,IO04,2017-12-09 12:00:00,2017,Atlantic,30,1000.0,1,193,873,110,34,124,100,259,99,65,198,3400,67,53,35,664,17.0,23.0,30,32,-1,-5,255,67,196,870,70.0,-3,1,3,34,63,-1,-46,27,1,126,151,17,287,20.0,-668,-524,-407,201,25,77,63,287,106,69,114,26,-1,-4,103,-1
1377,IO042017,IO04,2017-12-09 18:00:00,2017,Atlantic,25,1009.0,1,198,876,111,32,118,91,257,163,142,184,3359,68,52,32,674,17.0,23.0,31,-1,1,-4,236,72,200,876,65.0,-2,1,3,24,62,-17,-53,32,0,150,181,41,295,-2.0,-665,-525,-408,229,44,112,58,310,127,66,162,37,-22,-37,107,-1
1378,IO042017,IO04,2017-12-10 00:00:00,2017,Atlantic,25,1009.0,1,204,881,115,31,115,74,253,252,209,138,3337,70,53,33,699,19.0,28.0,24,-25,3,-3,224,76,200,880,52.0,-2,2,2,21,54,-15,-78,37,1,149,207,62,319,-31.0,-662,-523,-408,279,66,139,73,364,130,61,201,35,-6,-16,114,-1
1379,IO042017,IO04,2017-12-10 06:00:00,2017,Atlantic,20,1007.0,1,211,887,119,32,40,60,250,281,270,106,3405,68,50,28,712,26.0,33.0,11,-25,2,-2,259,65,209,883,41.0,-1,1,0,24,37,-25,-112,41,5,150,248,78,368,-21.0,-660,-522,-409,273,78,128,82,367,139,54,90,24,-2,-29,65,-1


In [118]:
# Step 2: Prepare Data

# Shuffle all unique hurricane IDs
np.random.seed(0)
ids_lst = df_data['ID'].unique()
np.random.shuffle(ids_lst)

# Split Data
train_data, val_data, test_data, train_ids, val_ids, test_ids = split_data(df_data, ids_lst, 0.6, 0.2, 0.2)
print('Training Data:', train_data.shape)
print('Validation Data:', val_data.shape)
print('Test Data:', test_data.shape)

# Normalize Data
scaler = Norm()
train_data_n, val_data_n, test_data_n = scaler.normalize_data(train_data, val_data, test_data)

Training Data: (819, 64)
Validation Data: (307, 64)
Test Data: (254, 64)


In [119]:
print(test_ids)

['IO042005' 'IO042006' 'IO022011' 'IO032006' 'IO032009' 'IO022015'
 'IO062013' 'IO022004' 'IO062008' 'IO012006' 'IO022003' 'IO012017'
 'IO052015' 'IO012015' 'IO032011' 'IO052010']


In [120]:
# Step 3: Prepare input and output data
train_inputs, train_labels = get_model_data(train_ids, train_data_n)
val_inputs, val_labels = get_model_data(val_ids, val_data_n)
test_inputs, test_labels = get_model_data(test_ids, test_data_n)
test_inputs = test_inputs[:-1]
test_labels = test_labels[:-1]
print('Train_inputs:', train_inputs.shape)
print('Train_labels', train_labels.shape)
print('Val_inputs:', val_inputs.shape)
print('Val_labels', val_labels.shape)
print('Test_inputs:', test_inputs.shape)
print('Test_labels', test_labels.shape)

Train_inputs: (45, 89, 55)
Train_labels (45, 89, 2)
Val_inputs: (15, 89, 55)
Val_labels (15, 89, 2)
Test_inputs: (15, 89, 55)
Test_labels (15, 89, 2)


In [121]:
# Step 4: Convert Data Type
train_inputs = train_inputs.astype(np.float32)
train_labels = train_labels.astype(np.float32)
val_inputs = val_inputs.astype(np.float32)
val_labels = val_labels.astype(np.float32)
test_inputs = test_inputs.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [122]:
# Step 5: Prepare GRU model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# create feature and targets tensor for train set. 
train_inputs_t = torch.from_numpy(train_inputs).cpu()
train_labels_t = torch.from_numpy(train_labels).cpu()

val_inputs_t = torch.from_numpy(val_inputs).cpu()
val_labels_t = torch.from_numpy(val_labels).cpu()

# create feature and targets tensor for test set.
test_inputs_t = torch.from_numpy(test_inputs).cpu()
test_labels_t = torch.from_numpy(test_labels).cpu()

# batch_size, epoch and iteration
batch_size = 90
num_epochs = 100000

# Pytorch train and test sets
train = TensorDataset(train_inputs_t, train_labels_t)
val = TensorDataset(val_inputs_t, val_labels_t)
test = TensorDataset(test_inputs_t, test_labels_t)

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
val_loader = DataLoader(val, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)

In [123]:
# Step 6: Define GRU Model

# Define dimensions
input_dim = train_inputs_t.size()[2]
hidden_dim = 55
layer_dim = 1
output_dim = train_labels_t.size()[2]
seq_dim = train_inputs_t.size()[1]
model = GRU_Model(input_dim, hidden_dim, layer_dim, output_dim)
model.cpu()

# Instantiate Loss Class
# criterion = nn.CrossEntropyLoss()
mse_loss_func = nn.MSELoss()

# Instantiate Optimizer Class
learning_rate = 0.65
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01, centered=True)
# optimizer = torch.optim.AdaDelta(model.parameters())
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=200, verbose=True)

In [124]:
# Step 7: LSTM_Handler Initializer
# self.model = model
# self.loss_fn = loss_fn
# self.optimizer = optimizer
# self.scheduler = scheduler
# self.train_losses = []
# self.val_losses = []

gru_handler = GRU_Handler(model, mse_loss_func, optimizer, scheduler)
gru_handler.train(train_loader, val_loader, batch_size, num_epochs, input_dim, seq_dim)

RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor

In [ ]:
lstm_handler.plot_losses()

In [ ]:
# Much higher loss because latitude and longitude outputs are being used as part of next inputs

actual3d, predicted3d, test_loss = lstm_handler.evaluate(test_loader, seq_dim, feed_forward=True)
test_loss

In [ ]:
plt.plot(lstm_handler.train_losses[2000:], label="Training loss")
plt.plot(lstm_handler.val_losses[2000:], label="Validation loss")
plt.legend()
plt.title("Losses for Training and Validation\n")

In [ ]:
plt.plot(lstm_handler.distances[:], label="Mean Distance Error")
plt.legend()
plt.title("Mean Distance Error for Training Dataset\n")

In [ ]:
filename = './data/lstm_cell_model.pkl'
lstm_handler.save_model(filename)

In [ ]:
# idx = np.where(test_ids == "AL052012")[0].item()
idx=51
print(idx)
id2 = test_ids[idx]
print(id2)
storm_len = df_data.loc[df_data.ID == id2].shape[0]
origin = df_data.loc[df_data.ID == id2][['LAT', 'LON']].iloc[0] / 10
lat_pred = predicted3d[idx, :storm_len-1, 0]
lon_pred = predicted3d[idx, :storm_len-1, 1]*-1
lat_actual = actual3d[idx, :storm_len-1, 0]
lon_actual = actual3d[idx, :storm_len-1, 1]*-1

lat_pred = np.insert(lat_pred, 0, origin[0])
lon_pred = np.insert(lon_pred, 0, origin[1]*-1)
lat_actual = np.insert(lat_actual, 0, origin[0])
lon_actual = np.insert(lon_actual, 0, origin[1]*-1)

In [ ]:
hurricane_map = map_hurricane("Hurricane Prediction")
lat_storm = lat_pred
lon_storm = lon_pred
x1, y1 = hurricane_map(lon_storm,lat_storm)
x2, y2 = hurricane_map(lon_actual, lat_actual)

#Prediction
plt.plot(x1,y1,'-',linewidth=2,color='red', label="Predicted")

#Actual
plt.plot(x2,y2,'-',linewidth=2,color='blue', label="Actual")

plt.legend(loc="lower right")

In [ ]:
idx = 0
storm_lengths = []
for x in range(len(test_ids)):
    id2 = test_ids[idx]
    storm_len = df_data.loc[df_data.ID == id2].shape[0]
    storm_lengths.append(storm_len)
    idx += 1

In [ ]:
df_testlen = pd.DataFrame(storm_lengths)
df_testlen = df_testlen.sort_values(by=[0])

In [ ]:
for index in np.array(df_testlen.tail(9).index):
    print(index, end=' ')
    idx=index
    id2 = test_ids[idx]
    name = df_data.loc[df_data.ID == id2]['NAME'].iloc[0]
    year = df_data.loc[df_data.ID == id2]['YEAR'].iloc[0]
    storm_len = df_data.loc[df_data.ID == id2].shape[0]
    print(name, year, storm_len)
    origin = df_data.loc[df_data.ID == id2][['LAT', 'LON']].iloc[0] / 10
    lat_pred = predicted3d[idx, :storm_len-1, 0]
    lon_pred = predicted3d[idx, :storm_len-1, 1]*-1
    lat_actual = actual3d[idx, :storm_len-1, 0]
    lon_actual = actual3d[idx, :storm_len-1, 1]*-1

    lat_pred = np.insert(lat_pred, 0, origin[0])
    lon_pred = np.insert(lon_pred, 0, origin[1]*-1)
    lat_actual = np.insert(lat_actual, 0, origin[0])
    lon_actual = np.insert(lon_actual, 0, origin[1]*-1)
    
    hurricane_map = map_hurricane("Storm Path for " + name)
    lat_storm = lat_pred
    lon_storm = lon_pred
    x1, y1 = hurricane_map(lon_storm,lat_storm)
    x2, y2 = hurricane_map(lon_actual, lat_actual)

    #Prediction
    plt.plot(x1,y1,'-',linewidth=2,color='red', label="Predicted")

    #Actual
    plt.plot(x2,y2,'-',linewidth=2,color='blue', label="Actual")

    plt.legend(loc="lower right")

In [ ]:
with torch.no_grad():
    batch = 0
    targets = []
    for data, target in train_loader:
        target2 = target.view(-1, 2)
        target2 = scaler.unnormalize(target2.cpu().detach().numpy())
        target2 /= 10
        target2 = np.reshape(target2, (-1, 89, 2))
        targets.append(target2)
    for data, target in val_loader:
        target2 = target.view(-1, 2)
        target2 = scaler.unnormalize(target2.cpu().detach().numpy())
        target2 /= 10
        target2 = np.reshape(target2, (-1, 89, 2))
        targets.append(target2)
    for data, target in test_loader:
        target2 = target.view(-1, 2)
        target2 = scaler.unnormalize(target2.cpu().detach().numpy())
        target2 /= 10
        target2 = np.reshape(target2, (-1, 89, 2))
        targets.append(target2)

In [ ]:
targets = np.concatenate(targets, axis=0)

In [ ]:
ids = np.concatenate([train_ids, val_ids, test_ids], axis=0)

In [ ]:
from matplotlib.lines import Line2D

def color_fn(cat):
    if cat == -1:
        return '#8dcdff'
    elif cat == 0:
        return '#fa8f8b'
    elif cat == 1:
        return '#009aff'
    elif cat == 2:
        return '#ff0303'
    elif cat == 3:
        return '#0278ee'
    elif cat == 4:
        return '#c60000'
    elif cat == 5:
        return '#1446bd'

hurricane_map = map_hurricane("Atlantic Hurricane Trajectories")

idx = 0
for hurricane in targets:
    id2 = ids[idx]
    storm_len = df_data.loc[df_data.ID == id2].shape[0]
    cat = max(df_data.loc[df_data.ID == id2]['CATEGORY'])
    x2, y2 = hurricane_map(hurricane[:storm_len-1, 1]*-1, hurricane[:storm_len-1, 0])
    plt.plot(x2,y2,'-',linewidth=2,color=color_fn(cat), label="Actual")
    idx += 1


custom_lines = [Line2D([0], [0], color='#8dcdff', lw=16),
                Line2D([0], [0], color='#fa8f8b', lw=16),
                Line2D([0], [0], color='#009aff', lw=16),
                Line2D([0], [0], color='#ff0303', lw=16),
                Line2D([0], [0], color='#0278ee', lw=16),
                Line2D([0], [0], color='#c60000', lw=16),
                Line2D([0], [0], color='#1446bd', lw=16)]

plt.legend(custom_lines, ['Tropical Depression', 'Tropical Storm', 'Category 1',
            'Category 2', 'Category 3', 'Category 4', 'Category 5'], prop={'size': 24}, loc='upper left')